In [1]:
# following code  imports the BigQuery Python Client Library and initialize a client. 
# The BigQuery client will be used to send and receive messages from the BigQuery API.
from google.cloud import bigquery
client = bigquery.Client()

sql = """
SELECT start_station_name, COUNT(*) AS num FROM 
`bigquery-public-data.london_bicycles.cycle_hire` GROUP BY start_station_name ORDER BY start_station_name;
"""
df = client.query(sql).to_dataframe()
df.head()

# now that the bigquery connection is demonstrably working (to the public data dataset)
# lets look at using the langchain LLM to convert questions/prompts into SQL queries
# and query the data using english language questions

#https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/orchestration/langchain/langchain_bigquery_data_loader.ipynb#scrollTo=AVdMAVibsvWz


,start_station_name,num
0,"Abbey Orchard Street, Westminster",121574
1,"Abbotsbury Road, Holland Park",38191
2,"Aberdeen Place, St. John's Wood",61413
3,"Aberfeldy Street, Poplar",21158
4,"Abingdon Green, Great College Street",4669


In [7]:

# directions for setting up the google VERTICA workbench environment
#https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env
#!pip install google-cloud-aiplatform --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 20.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.58.0
    Uninstalling google-cloud-aiplatform-1.58.0:
      Successfully uninstalled google-cloud-aiplatform-1.58.0


In [2]:
!pip install langchain==0.0.332

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.0 MB/s eta 0:00:00


In [3]:
#import google.cloud.bigquery as bq
import langchain
from google.cloud import aiplatform
from langchain.llms import VertexAI
from langchain.document_loaders import BigQueryLoader
from langchain.prompts import PromptTemplate
from langchain.schema import format_document

# Print LangChain and Vertex AI versions
print(f"LangChain version: {langchain.__version__}")
print(f"Vertex AI SDK version: {aiplatform.__version__}")

LangChain version: 0.0.332
Vertex AI SDK version: 1.58.0


In [4]:
#You will need to initialize vertexai with your project_id and location:

PROJECT_ID = "qwiklabs-gcp-00-75d1d8733508"
LOCATION = "us-central1"

import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [5]:
llm = VertexAI(model_name="text-bison", temperature=0)
#llm = VertexAI(model_name="code-bison@latest", max_output_tokens=2048)
llm("What's BigQuery?")

' BigQuery is a cloud-based, serverless, highly scalable, and cost-effective data warehouse solution offered by Google Cloud Platform (GCP). It allows businesses and organizations to store, process, and analyze large amounts of data in a fast and efficient manner. Here are some key features and benefits of BigQuery:\n\n**1. Scalability:** BigQuery is designed to handle massive datasets, ranging from terabytes to petabytes, without compromising on performance. It automatically scales up or down based on the workload, ensuring that queries are processed quickly and efficiently.\n\n**2. Serverless Architecture:** BigQuery operates on a'

In [6]:
# Define our query
# specifically targeting meta data of the tables involved
# this pulls back all of the metadata for the tables in the database in question
# which the 'chain' object can use to help the model generate queries from english language statements
query = f"""
SELECT table_name, ddl
FROM `bigquery-public-data.london_bicycles.INFORMATION_SCHEMA.TABLES`
WHERE table_type = 'BASE TABLE'
ORDER BY table_name;
"""
# Load the data
loader = BigQueryLoader(
    query, metadata_columns="table_name", page_content_columns="ddl"
)
# create a 'load' object from the BigQueryLoader function
data = loader.load()



In [44]:
# we will get the prompt specific input from a streamlit input box
#issue = st.text_input("Enter the medical issue description:")
querypromptcontent  = "help me count bike journeys by start location"

In [46]:
# Use code generation model
#llm = VertexAI(model_name="code-bison@latest", max_output_tokens=2048)
llm = VertexAI(model_name="text-bison", temperature=0)

# Define the chain
chain = (
    {
        "content": lambda docs: "\n\n".join(
            format_document(doc, PromptTemplate.from_template("{page_content}"))
            for doc in docs
        )
    }
    | PromptTemplate.from_template(
        f"""Suggest a GoogleSQL query to {querypromptcontent}:\n\n{{content}}"""
    )
    | llm
)

# Invoke the chain with the documents, and remove code backticks
# chain object (combination of user prompt and llm model) uses metadata to dynamically create query
generatedquery = chain.invoke(data).strip("```")
generatedquery = generatedquery[7:10000]
#generatedquery =  f'"""{generatedquery}"""'
#print(generatedquery)

# need to put some error trapping around the query definition because sometimes following on from the
# user prompt the ai writes the query out incorrectly - i.e. it gets the field names incorrect at times!
df2 = client.query(generatedquery).to_dataframe()
df2.head()



,start_station_id,bike_journeys
0,191,658129
1,14,579703
2,154,512990
3,303,451863
4,307,451696


NameError: name 'bq' is not defined